In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Dicionário com as abreviações dos meses em inglês
meses_abreviados = {
    "january": "Jan",
    "february": "Feb", 
    "march": "Mar",
    "april": "Apr",
    "may": "May",
    "june": "Jun",
    "july": "Jul",
    "august": "Aug",
    "september": "Sep",
    "october": "Oct",
    "november": "Nov",
    "december": "Dec"
}

# PJs 
pjs_verificar = {
    "PJ - Anderson Julimar Breger Cenci": {"campo1": False, "campo2": False},
    "PJ - Daniel Prestes da Silveira": {"campo1": False, "campo2": False},
    "PJ - Gustavo Luiz Presa": {"campo1": False, "campo2": False},
    "PJ - Marcos Antônio Carvalho": {"campo1": False, "campo2": False},
    "PJ - Marlon Pohl": {"campo1": False, "campo2": False},
    "PJ - Matheus Barreiros Michelotti": {"campo1": False, "campo2": False},
    "PJ - Paulo Cabreira": {"campo1": False, "campo2": False},
    "PJ - Sidênio Samuel Quarti": {"campo1": False, "campo2": False},
    "Exitum": {"campo1": False, "campo2": False}
}

try:
    # Calcula o mês desejado: dois meses antes da data atual
    data_alvo = datetime.now() - relativedelta(months=2)
    mes_alvo = data_alvo.strftime("%B").lower()
    ano_alvo = data_alvo.strftime("%Y")
    mes_abreviado = meses_abreviados.get(mes_alvo)

    if not mes_abreviado:
        print(f"\nMês não encontrado no dicionário: {mes_alvo}")
        exit()

    driver = webdriver.Chrome()
    driver.get("https://osprdfar01.ad-grendene.com/FaturamentoTerceirosGrendene/ConsultaGovernanca.aspx")
    wait = WebDriverWait(driver, 20)

    # Clica no seletor do campo do mês
    seletor_xpath = "//span[contains(@class, 'select2-selection') and contains(@id, 'Periodo-container') or @role='combobox']"
    campo_seletor = wait.until(EC.element_to_be_clickable((By.XPATH, seletor_xpath)))
    campo_seletor.click()
    time.sleep(1)

    # Monta o XPath para localizar a opção correta do mês
    xpath_opcao_mes = f"//li[contains(text(), '{mes_abreviado} (Vigência de')]"
    opcao_mes = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_opcao_mes)))
    print(f"Selecionando: {opcao_mes.text}")
    opcao_mes.click()

    time.sleep(2)

    # Confere o valor que foi renderizado após a seleção
    render_xpath = "//span[contains(@class,'select2-selection__rendered')]"
    selecao_atual = driver.find_element(By.XPATH, render_xpath).text
    if mes_abreviado not in selecao_atual:
        print(f"\nErro: Mês não selecionado corretamente. Selecionado: {selecao_atual}")
        driver.quit()
        exit()
    
    print(f"Mês selecionado com sucesso: {selecao_atual}")

    # Selecionar o campo de pesquisa
    seletor_xpath = '//*[@id="LGrendeneTraditionalTheme_wt24_block_OutSystemsUIWeb_wt3_block_wtContent_wtMainContent_wtbtnGerar"]'
    botao_pesquisar = wait.until(EC.element_to_be_clickable((By.XPATH, seletor_xpath)))
    botao_pesquisar.click()

    time.sleep(3)

    # Verificação dos PJs específicos
    print("\nVerificando campos dos PJs...")
    rows = driver.find_elements(By.CSS_SELECTOR, "tr")
    
    for row in rows:
        try:
            nome_elemento = row.find_element(By.CSS_SELECTOR, "td:first-child")
            nome_pj = nome_elemento.text.strip()
            
            for pj in pjs_verificar:
                if pj.lower() in nome_pj.lower():
                    campos_valor = row.find_elements(By.CSS_SELECTOR, ".input.OSFillParent")
                    
                    if len(campos_valor) >= 2:
                        valor1 = campos_valor[0].get_attribute("value")
                        valor2 = campos_valor[1].get_attribute("value")

                        if valor1 and float(valor1.replace(',', '.')) > 0:
                            pjs_verificar[pj]["campo1"] = True
                        
                        if valor2 and float(valor2.replace(',', '.')) > 0:
                            pjs_verificar[pj]["campo2"] = True

                        print(f"\nPJ: {pj}")
                        print(f"  Campo 1: {valor1 if valor1 else 'Não preenchido'}")
                        print(f"  Campo 2: {valor2 if valor2 else 'Não preenchido'}")
        except Exception as e:
            continue

    # Verifica se todos os PJs têm ambos os campos preenchidos
    pjs_incompletos = []
    for pj, status in pjs_verificar.items():
        if not (status["campo1"] and status["campo2"]):
            campos_faltantes = []
            if not status["campo1"]: campos_faltantes.append("Campo 1")
            if not status["campo2"]: campos_faltantes.append("Campo 2")
            pjs_incompletos.append(f"- {pj}: {', '.join(campos_faltantes)} não preenchido(s)")

    if pjs_incompletos:
        print("\nVerificação incompleta:")
        print("\n".join(pjs_incompletos))
        print("\nOperação cancelada: Alguns campos precisam ser preenchidos.")
        driver.quit()
        exit()

   
    # Clica no botão de exportação
    botao_exportar_xpath = '//*[@id="LGrendeneTraditionalTheme_wt24_block_OutSystemsUIWeb_wt3_block_wtContent_wtMainContent_wt92"]/span[2]'
    botao_exportar = wait.until(EC.element_to_be_clickable((By.XPATH, botao_exportar_xpath)))

    if botao_exportar.is_displayed() and botao_exportar.is_enabled():
        botao_exportar.click()
        print("\nExportação iniciada com sucesso.")
        time.sleep(5)
    else:
        print("\nBotão de exportação não está clicável.")
        driver.quit()
        exit()

except Exception as e:
    print(f"\nErro inesperado: {e}")
""""
finally:
    driver.quit()
    print("\nNavegador fechado.")
    """


Erro inesperado: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x00007FF73C36CF65+75717]
	GetHandleVerifier [0x00007FF73C36CFC0+75808]
	(No symbol) [0x00007FF73C138F9A]
	(No symbol) [0x00007FF73C111B41]
	(No symbol) [0x00007FF73C1BEC2E]
	(No symbol) [0x00007FF73C1DECC2]
	(No symbol) [0x00007FF73C1B7153]
	(No symbol) [0x00007FF73C180421]
	(No symbol) [0x00007FF73C1811B3]
	GetHandleVerifier [0x00007FF73C66D74D+3223469]
	GetHandleVerifier [0x00007FF73C667CF2+3200338]
	GetHandleVerifier [0x00007FF73C685B23+3322755]
	GetHandleVerifier [0x00007FF73C386A3A+180890]
	GetHandleVerifier [0x00007FF73C38E13F+211359]
	GetHandleVerifier [0x00007FF73C3752B4+109332]
	GetHandleVerifier [0x00007FF73C375462+109762]
	GetHandleVerifier [0x00007FF73C35BA79+4825]
	BaseThreadInitThunk [0x00007FFF5D81257D+29]
	RtlUserThreadStart [0x00007FFF5F06AF08+40]



'"\nfinally:\n    driver.quit()\n    print("\nNavegador fechado.")\n    '